# Fazendo as inferências nas imagens

In [38]:
import multiprocessing as mp
import geopandas as gpd
from sklearnex import patch_sklearn
patch_sklearn()
import rasterio
from rasterio import features
from rasterio.enums import Resampling
import numpy as np
import glob
import joblib
from skimage import filters
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import geopandas as gpd
from shapely.geometry import box, MultiPolygon
from sklearn.svm import SVC
from os.path import exists


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [63]:
gdf_scms = gpd.read_file('downloads/SIRGAS_SHP_quadriculaortofoto2017.zip!SIRGAS_SHP_quadriculaortofoto2017/SIRGAS_SHP_quadriculaortofoto2017.shp')

In [40]:
# n_cpu = mp.cpu_count() - 1
n_cpu = 4

In [41]:
# sample =  gdf_scms[2:17]['qo2010_cod'].to_list()

In [42]:
# len(sample)

In [43]:
# sample

In [44]:
radius = 3
n_points = 8 * radius
tamanho_janela = 24 # tamanho em pixels
sobreposicao_de_janelas = 12 # sobreposição das janelas em pixels

In [45]:
gdf_quadras = gpd.read_file('downloads/SIRGAS_GPKG_quadraviariaed_2017.gpkg')

In [46]:
grayscale_scaler = joblib.load('pre-process/grayscale.scaler.save')
sobel_scaler = joblib.load('pre-process/sobel.scaler.save') 
lbp_scaler = joblib.load('pre-process/lbp.scaler.save')

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [47]:
pca = joblib.load('resultados/modelos-treinados/PCA-faixas-pedestre.pkl')

In [48]:
clf = joblib.load('resultados/modelos-treinados/SVC-faixas-pedestre.pkl')

In [66]:
len(glob.glob(f'../ortofotos-2017/RGB-2017/**.jp2'))

4250

In [49]:
def faixas(scm):
    if exists(f'resultados/raster/faixas-{scm}.jp2'):
        return None
    else:
        print(scm + " / ")
        file = glob.glob(f'../ortofotos-2017/RGB-2017/*{scm}*.jp2')
        dataset = rasterio.open(file[0])
        data, location = [], []

        grayscale = 0.2125 * dataset.read(1) + \
                0.7154 * dataset.read(2) + \
                0.0721 * dataset.read(3)
        grayscale_uint = grayscale.astype('uint8')

        grayscale = grayscale_scaler.transform(grayscale_uint.reshape(-1, 1)).reshape(grayscale_uint.shape).astype('float32')

        sobel = filters.sobel(grayscale_uint).astype('float32')
        sobel = sobel_scaler.transform(sobel.reshape(-1, 1)).reshape(sobel.shape)

        lbp = local_binary_pattern(grayscale_uint, n_points, radius).astype('float32')
        lbp = lbp_scaler.transform(lbp.reshape(-1, 1)).reshape(lbp.shape)

        quadras_scm = gpd.clip(gdf_quadras, box(*dataset.bounds), keep_geom_type=True).unary_union

        matrix = np.zeros([dataset.height, dataset.width], dtype='uint8')

        profile = dataset.profile
        profile['count'] = 1
        del profile['tiled']
        del profile['interleave']

        if quadras_scm is None:
            with rasterio.open(f'resultados/raster/faixas-{scm}.jp2', 'w', nbits=1, **profile) as dst:
                dst.write(matrix, indexes=1)
            return None

        if quadras_scm.geom_type == 'Polygon':
            quadras = features.rasterize(
                ((g, 255) for g in [quadras_scm]),
                out_shape=dataset.shape,
                transform=dataset.transform)
        else:
            quadras = features.rasterize(
                ((g, 255) for g in quadras_scm),
                out_shape=dataset.shape,
                transform=dataset.transform)

        for i in np.arange(0, grayscale.shape[0] - tamanho_janela, tamanho_janela - sobreposicao_de_janelas):
            for j in np.arange(0, grayscale.shape[1] - tamanho_janela, tamanho_janela - sobreposicao_de_janelas):
                # Se mais de 98% dos pixels estiverem fora da quadra
                if np.count_nonzero(quadras[i:i+tamanho_janela, j:j+tamanho_janela] == 255) < 0.02 * (tamanho_janela ** 2):
                    # print(i, j)
                    img_gray = grayscale[i:i+tamanho_janela, j:j+tamanho_janela]
                    img_sobel = sobel[i:i+tamanho_janela, j:j+tamanho_janela]
                    img_lbp = lbp[i:i+tamanho_janela, j:j+tamanho_janela]
                    
                    img = np.concatenate([
                        img_gray,
                        img_sobel,
                        img_lbp
                    ]).reshape(1, 3 * (tamanho_janela ** 2))
            
                    data.append(img.reshape(1, 3 * (tamanho_janela ** 2)))
                    location.append([i, j])

        np_data = np.array(data).squeeze()

        if len(np_data) > 0:
            # print(np_data.shape)
            # if quadras_scm.geom_type == 'Polygon':
            if np_data.ndim < 2:
                np_data = pca.transform([np_data])
            else:
                np_data = pca.transform(np_data)

            y = clf.predict(np_data)            

            for i, l in enumerate(location):
                matrix[l[0]:l[0] + tamanho_janela, l[1]:l[1] + tamanho_janela] = y[i]
            
            with rasterio.open(f'resultados/raster/faixas-{scm}.jp2', 'w', nbits=1, **profile) as dst:
                dst.write(matrix, indexes=1)

            return profile
        else:
            print(f'{scm} vazio!!')
            with rasterio.open(f'resultados/raster/faixas-{scm}.jp2', 'w', nbits=1, **profile) as dst:
                dst.write(matrix, indexes=1)
            # print(profile)
            return None

In [50]:
# y, location, data = faixas('3313-432')
# faixas('3335-312')

In [51]:
# with mp.Pool(processes=n_cpu) as p:
#     p.map(faixas, sample)

## Processando um distrito inteiro

In [52]:
gdf_distritos = gpd.read_file('downloads/SIRGAS_GPKG_distrito.gpkg')

In [53]:
# dist = gdf_distritos[gdf_distritos.ds_codigo == '90']
distritos = gdf_distritos

In [54]:
distritos

,ds_codigo,ds_subpref,ds_cd_sub,ds_sigla,ds_nome,geometry
0,63,PIRITUBA-JARAGUA,02,PIR,PIRITUBA,"POLYGON ((325109.668 7405122.828, 325106.786 7..."
1,95,PIRITUBA-JARAGUA,02,SDO,SAO DOMINGOS,"POLYGON ((319854.036 7402894.086, 319864.990 7..."
2,42,PIRITUBA-JARAGUA,02,JAR,JARAGUA,"POLYGON ((319172.403 7405179.204, 319174.204 7..."
3,11,FREGUESIA-BRASILANDIA,03,BRL,BRASILANDIA,"POLYGON ((324895.091 7404257.969, 324893.588 7..."
4,29,FREGUESIA-BRASILANDIA,03,FRE,FREGUESIA DO O,"POLYGON ((327469.803 7399113.522, 327449.180 7..."
...,...,...,...,...,...,...
91,44,SAO MIGUEL,23,JDH,JARDIM HELENA,"POLYGON ((352915.621 7402621.049, 352944.248 7..."
92,24,ITAQUERA,27,CLD,CIDADE LIDER,"POLYGON ((345181.289 7394204.666, 345180.470 7..."
93,57,ITAQUERA,27,PQC,PARQUE DO CARMO,"POLYGON ((352597.619 7390119.444, 352588.233 7..."
94,47,ITAQUERA,27,JBO,JOSE BONIFACIO,"POLYGON ((354091.203 7396702.971, 354091.662 7..."


In [60]:
for id, dist in distritos.iterrows():

    # scms = gdf_scms[gdf_scms.intersects(dist.geometry)]['qo2010_cod'].to_list()
    scms = gdf_scms['qo2010_cod'].to_list()
    # [faixas(scm) for scm in scms]
    with mp.Pool(processes=n_cpu) as p:
        p.map(faixas, scms)

2346-351 / 3441-342 / 3426-331 / 2443-263 / 



2443-432 / 
3336-453 / 
2346-354 / 
4436-262 / 
3443-462 / 
3431-451 / 
2441-451 / 
3441-344 / 
3426-313 / 
3443-464 / 
3445-232 / 
3426-362 / 
2446-111 / 
4436-134 / 
4436-252 / 
3426-364 / 
3442-131 / 
3441-211 / 
4436-133 / 
3441-134 / 
3432-364 / 
3432-354 / 
3432-343 / 
3431-454 / 
2446-414 / 
3444-111 / 
4445-342 / 
3441-133 / 
2442-234 / 
2442-233 / 
4436-241 / 
2442-223 / 
3434-232 / 
2442-242 / 
3433-232 / 
2446-454 / 
3441-154 / 
3433-231 / 
2443-222 / 
4436-141 / 
2442-243 / 
3441-321 / 
3431-411 / 
3444-113 / 
2442-331 / 
2446-453 / 
4321-121 / 
3434-224 / 
3446-251 / 
2322-221 / 
3434-223 / 
4321-123 / 
2446-162 / 
3434-214 / 
2446-152 / 
4312-433 / 
3434-213 / 
2446-142 / 
2443-252 / 
2324-222 / 
4436-253 / 
2324-224 / 
2322-223 / 
4321-151 / 
4314-252 / 
4312-452 / 
4435-264 / 
2324-253 / 
2324-343 / 
4325-113 / 
4435-263 / 
2326-254 / 
2326-424 / 
2326-323 / 
4316-422 / 
2324-244 / 
4314-434 / 
2326-314 / 
2326-313 / 
3326

In [62]:
gdf_scms.columns

Index(['qo2010_cod', 'geometry'], dtype='object')